In [1]:
import pandas as pd
import numpy as np
import os
import nltk 
nltk.download('vader_lexicon')
nltk.download('tagsets')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 


import re
from pprint import pprint
from statistics import mean

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import Phrases

from gensim.models.wrappers import LdaMallet

import os
os.environ["MALLET_HOME"]='C:\\Users\\mezhang\\AppData\\Local\\Anaconda\\mallet-2.0.8'
mallet_path =r"C:\Users\mezhang\AppData\Local\Anaconda\mallet-2.0.8\bin\mallet"

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\mezhang\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package tagsets to
[nltk_data]     C:\Users\mezhang\AppData\Roaming\nltk_data...
[nltk_data]   Package tagsets is already up-to-date!


In [2]:
os.getcwd()

'C:\\Users\\mezhang\\Desktop\\FB Comment Analysis'

## Instruction


https://medium.com/analytics-vidhya/simplifying-social-media-sentiment-analysis-using-vader-in-python-f9e6ec6fc52f
https://www.pingshiuanchua.com/blog/post/simple-sentiment-analysis-python?utm_campaign=News&utm_medium=Community&utm_source=DataCamp.com

The compound score is computed by summing the valence scores of each word in the lexicon, 
adjusted according to the rules, and then normalized to be between -1 (most extreme negative) 
and +1 (most extreme positive). This is the most useful metric if you want a single unidimensional 
measure of sentiment for a given sentence. Calling it a 'normalized, weighted composite score' is accurate.

It is also useful for researchers who would like to set 
standardized thresholds for classifying sentences as either positive, neutral, or negative. 
Typical threshold values (used in the literature cited on this page) are:

    positive sentiment: compound score >= 0.05
    neutral sentiment: (compound score > -0.05) and (compound score < 0.05)
    negative sentiment: compound score <= -0.05

The pos, neu, and neg scores are ratios for proportions of text 
that fall in each category (so these should all add up to be 1... or close to it with float operation). 
These are the most useful metrics if you want multidimensional measures of sentiment for a given sentence.


In [3]:
# transform the input table to create the list of postid for fetching data in facepager
def input_preparetion(file_name):
    xlsx_file=file_name+'.xlsx'
    postid=pd.read_excel(xlsx_file,sheet_name='Sheet1',dtype=object)
    postid['Postid']=postid['URL'].str.split('/',expand=True).iloc[:,-1]
    postid['Section']=postid['Final Link'].str.split('/',expand=True).iloc[:,3]
    postid['Subsection']=postid['Final Link'].str.split('/',expand=True).iloc[:,4]
    postid=postid.drop([ 'URL','Link'], axis=1)
    postid=postid.rename(columns={'Message':'Post_Headline'})
    postid['Fullid']= postid['Page Id'].astype(str)+'_'+postid['Postid'].astype(str)
    postid = postid[['Page Id', 'Postid', 'Fullid', 'Post_Headline','Final Link','Section','Subsection']]
    return postid
    

In [7]:
score=sentiment_score('filefromrati_comments1')

C:\Users\mezhang\AppData\Local\Anaconda\lib\site-packages\defusedxml\ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__init__(self, html, target, encoding)
C:\Users\mezhang\AppData\Local\Anaconda\lib\site-packages\defusedxml\ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__init__(self, html, target, encoding)
C:\Users\mezhang\AppData\Local\Anaconda\lib\site-packages\defusedxml\ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__init__(self, html, target, encoding)
C:\Users\mezhang\AppData\Local\Anaconda\lib\site-packages\defusedxml\ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__init__(self, html, target, encoding)
C:\Users\mezhang\AppData\Local\Anaconda\lib\site-packages\defusedxml\ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__in

In [10]:
r=input_preparetion('filefromrati1')
combined=pd.merge(score,r,on='Postid')

C:\Users\mezhang\AppData\Local\Anaconda\lib\site-packages\defusedxml\ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__init__(self, html, target, encoding)
C:\Users\mezhang\AppData\Local\Anaconda\lib\site-packages\defusedxml\ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__init__(self, html, target, encoding)
C:\Users\mezhang\AppData\Local\Anaconda\lib\site-packages\defusedxml\ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__init__(self, html, target, encoding)
C:\Users\mezhang\AppData\Local\Anaconda\lib\site-packages\defusedxml\ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__init__(self, html, target, encoding)
C:\Users\mezhang\AppData\Local\Anaconda\lib\site-packages\defusedxml\ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__in

In [114]:
business=combined[combined['Section']=='news']
business['Article_Sentiment'] = np.where(business['Postive_Comments']<business['Negative_Comments'], 'N', 'P')

C:\Users\mezhang\AppData\Local\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [112]:
business

,Postid,Total_Comments,Postive_Comments,Neutral_Comments,Negative_Comments,NA_Comments,Page Id,Fullid,Post_Headline,Final Link,Section,Subsection,Article_Sentiment
5,10156954402858256,11,4,0,3,4,5953023255,5953023255_10156954402858256,Nick Foles is replacing Carson Wentz at quarte...,https://www.chicagotribune.com/sports/football...,sports,football,P
7,10156954909978256,7,2,0,1,4,5953023255,5953023255_10156954909978256,"A sellout crowd of 76,126 made this year's Win...",https://www.chicagotribune.com/sports/hockey/b...,sports,hockey,P
12,10156955965723256,6,2,1,2,1,5953023255,5953023255_10156955965723256,Northwestern pulled off a thrilling comeback v...,https://www.chicagotribune.com/sports/college/...,sports,college,P
13,10156956335538256,9,3,0,1,5,5953023255,5953023255_10156956335538256,Bears fans have waited 8 years for playoff foo...,https://www.chicagotribune.com/sports/football...,sports,football,P
14,10156956438498256,13,2,0,2,9,5953023255,5953023255_10156956438498256,Gene Okerlund became a stalwart of World Wrest...,https://www.chicagotribune.com/sports/breaking...,sports,breaking,P
43,10156960475488256,12,6,0,0,6,5953023255,5953023255_10156960475488256,"Last March, the Bears formed a nine-person soc...",https://www.chicagotribune.com/sports/football...,sports,football,P
47,10156960931588256,2,1,0,0,1,5953023255,5953023255_10156960931588256,Four Bears were named to the NFL's All-Pro fir...,https://www.chicagotribune.com/sports/football...,sports,football,P
50,10156961319553256,9,2,0,1,6,5953023255,5953023255_10156961319553256,Sunday marks the fourth time the two teams wil...,https://www.chicagotribune.com/sports/football...,sports,football,P
71,10156966193653256,51,21,1,12,17,5953023255,5953023255_10156966193653256,The Bears' remarkable turnaround season came t...,https://www.chicagotribune.com/sports/football...,sports,football,P
73,10156966505483256,45,17,0,15,13,5953023255,5953023255_10156966505483256,"The Bears, who proved naysayers wrong in a maj...",https://www.chicagotribune.com/sports/football...,sports,football,P


In [115]:
lda_top_words(business)

,Postid,Topwords_level1,Topwords_level2,Topwords_level3
0,10156953862688256,"Ramirez,Baby,Yarehlie,Hour,Girl,County,Wasn","Baby Born,Medical Center",
0,10156954215838256,"Boomer,Time,World,Generation,Baby,Sorry,Someone",,
0,10156954957663256,"Apl,Scientist,Mission,Thule,Ultima,Object,Spac...","Nasa Shows,New Horizons,Kuiper Belt,,Mission O...",New Horizons Spacecraft.
0,10156955515998256,"President,Kingdom,Country,Falwell,Think,Jesus,...","Liberty University,President Jerry,Falwell Jr....",
0,10156955851433256,"Trump,President,Party,Romney,Mitt,Donald,State","Donald Trump,Romney Leaves,Trump National,Golf...",White House Chief
0,10156956880198256,"Gay,Time,Illinois,Inmate,Confinement,State,Pri...","Anthony Gay,Illinois Prison,Illinois Departmen...",
0,10156957198658256,"Officer,Johnson,Police,Shot,Building,Gun,Apart...","Police Officer,Police Reports",
0,10156957429523256,"Munoz,Wife,Garcia,Wednesday,Year,Judge,Gugliel...","Wednesday Morning,Cook County,Guglielmi Said.,...",
0,10156957735948256,"Customer,Employee,Mcdonald,Man,Counter,Police,...","St. Petersburg,Plastic Straws,Biandudi Said.",
0,10156957818938256,"Ceremony,County,Clerk,Couple,Marriage,Office,L...",Life Event,


In [6]:
# sentiment score tool
def nltk_sentiment(sentence):
    from nltk.sentiment.vader import SentimentIntensityAnalyzer
    nltk_sentiment = SentimentIntensityAnalyzer()
    score = nltk_sentiment.polarity_scores(sentence)
    return score

# on article/postid level the sentiment of all comments
def sentiment_score(file_name):
    xlsx_file=file_name+'.xlsx'
    raw=pd.read_excel(xlsx_file,sheet_name='Sheet1',dtype=object)
    raw_wo_null=raw.dropna()
    dataset=raw_wo_null['message'].tolist()
    
    nltk_results = [nltk_sentiment(str(row)) for row in dataset]
    y=pd.DataFrame.from_dict(nltk_results)
    y.reset_index(drop=True, inplace=True)
    raw_wo_null.reset_index(drop=True, inplace=True)
    table=pd.concat([raw_wo_null,y],axis=1)
    table=table.rename(columns={'compound':'Compound_Score','neg': 'Negative_Score', 'neu': 'Neutral_Score','pos':'Postive_Score'})
    table['Postid']=table['object_id'].str.split('_',expand=True,).iloc[:,0]
    
    table['Negative_Comments'] = np.where(table['Compound_Score']<-0.05, 1, 0)
    table['Postive_Comments'] = np.where(table['Compound_Score']>0.05, 1, 0)
    table['Neutral_Comments'] = np.where((table['Compound_Score']<=0.05) & (table['Compound_Score']>= -0.05) & (table['Compound_Score'])!=0, 1, 0)
    table['NA_Comments'] = np.where((table['Compound_Score'])==0, 1, 0)
    summary=table.groupby('Postid',as_index=False).sum()
    summary['Total_Comments']=summary['Negative_Comments']+summary['Postive_Comments']+summary['Neutral_Comments']+summary['NA_Comments']
    summary=summary[['Postid','Total_Comments','Postive_Comments','Neutral_Comments','Negative_Comments','NA_Comments']]
    return summary


# on article/postid_comment level the sentiment of each comment
def comment_sentiment_score(file_name):
    text_file=file_name+'.csv'
    raw=pd.read_csv(text_file,sep=';')
    raw_wo_null=raw.dropna()
    dataset=raw_wo_null['message'].tolist()
    
    nltk_results = [nltk_sentiment(str(row)) for row in dataset]
    y=pd.DataFrame.from_dict(nltk_results)
    y.reset_index(drop=True, inplace=True)
    raw_wo_null.reset_index(drop=True, inplace=True)
    table=pd.concat([raw_wo_null,y],axis=1)
    table=table.rename(columns={'compound':'Compound_Score','neg': 'Negative_Score', 'neu': 'Neutral_Score','pos':'Postive_Score'})
    table['Postid']=table['object_id'].str.split('_',expand=True,).iloc[:,0]
    
    table['Negative_Comments'] = np.where(table['Compound_Score']<-0.05, 1, 0)
    table['Postive_Comments'] = np.where(table['Compound_Score']>0.05, 1, 0)
    table['Neutral_Comments'] = np.where((table['Compound_Score']<=0.05) & (table['Compound_Score']>= -0.05) & (table['Compound_Score'])!=0, 1, 0)
    table['NA_Comments'] = np.where((table['Compound_Score'])==0, 1, 0)
    table['Total_Comments']=table['Negative_Comments']+table['Postive_Comments']+table['Neutral_Comments']+table['NA_Comments']
    table=table[['Postid','message','Total_Comments','Postive_Comments','Neutral_Comments','Negative_Comments','NA_Comments']]
    return table


        
        
def export_to_file(postid_file,comment_file,to_file_name):
    postid=input_preparetion(postid_file)
    score=sentiment_score(comment_file)
    result = postid.merge(score, left_index=True, right_index=True,how='inner', suffixes=('', '_y'))
    result=result.drop(['Postid_y'],axis=1)
    xlsx_file=to_file_name+'.xlsx'
    result.to_excel(xlsx_file,sheet_name='Sheet1')
    return result

In [83]:
xlsx_file='filefromrati_comments1'+'.xlsx'
raw=pd.read_excel(xlsx_file,sheet_name='Sheet1',dtype=object)
raw_wo_null=raw[:20000].dropna()
dataset=raw_wo_null['message'].tolist()

C:\Users\mezhang\AppData\Local\Anaconda\lib\site-packages\defusedxml\ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__init__(self, html, target, encoding)
C:\Users\mezhang\AppData\Local\Anaconda\lib\site-packages\defusedxml\ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__init__(self, html, target, encoding)
C:\Users\mezhang\AppData\Local\Anaconda\lib\site-packages\defusedxml\ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__init__(self, html, target, encoding)
C:\Users\mezhang\AppData\Local\Anaconda\lib\site-packages\defusedxml\ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__init__(self, html, target, encoding)
C:\Users\mezhang\AppData\Local\Anaconda\lib\site-packages\defusedxml\ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__in

In [84]:


nltk_results = [nltk_sentiment(str(row)) for row in dataset]
y=pd.DataFrame.from_dict(nltk_results)
y.reset_index(drop=True, inplace=True)
raw_wo_null.reset_index(drop=True, inplace=True)
table=pd.concat([raw_wo_null,y],axis=1)
table=table.rename(columns={'compound':'Compound_Score','neg': 'Negative_Score', 'neu': 'Neutral_Score','pos':'Postive_Score'})
table['Postid']=table['object_id'].str.split('_',expand=True,).iloc[:,0]

table['Negative_Comments'] = np.where(table['Compound_Score']<-0.05, 1, 0)
table['Postive_Comments'] = np.where(table['Compound_Score']>0.05, 1, 0)
table['Neutral_Comments'] = np.where((table['Compound_Score']<=0.05) & (table['Compound_Score']>= -0.05) & (table['Compound_Score'])!=0, 1, 0)
table['NA_Comments'] = np.where((table['Compound_Score'])==0, 1, 0)
table['Total_Comments']=table['Negative_Comments']+table['Postive_Comments']+table['Neutral_Comments']+table['NA_Comments']
table=table[['Postid','message','Total_Comments','Postive_Comments','Neutral_Comments','Negative_Comments','NA_Comments']]



In [88]:
table[table['Postid']=='10157038204408256'].to_excel("relevant_example.xlsx",sheet_name='Sheet1')

In [91]:
c=pd.merge(table,r,on='Postid')
d=c[c['Section']=='business']
d[d['Negative_Comments']==1]['message'].to_list()

["...and stop calling me about my car's warranty expiring!",
 'They should warn about the scammers calling saying they are from Microsoft or Windows and they have discovered a problem with your computer!  They are relentless.  I keep them on the phone for as long as I have time pretending I’m an old confused person as I am “helping them” take control of my computer.  Just about the time I can imagine them licking their lips over the money they’re about to get from me, I tell them they picked the wrong pigeon and while it has been annoying, I saved two or three others by tying them up.  They hang up frustrated and furious.  And, yes, I’m on the no call list - again.',
 "My mom was getting these calls and tried to report them, but she was told there's nothing that can be done and to simply ignore them. Why does it take an attempt on a former FBI/CIA director to get any action on these scammers?",
 'Now if someone would only stop all these spam calls.',
 'I’m living this mess now.  My fat

In [107]:
#extract content from url using beautifulsoup
from bs4 import BeautifulSoup as bs
import requests

def converttolist (resultset):
    x=[]
    for line in resultset:
        x.append(line.text)
    return x

def content_from_url (url):
    page = requests.get(url)
    soup = bs(page.content, 'html.parser')
    p=soup.findAll("p")
    em=soup.find_all("em")
    bold=soup.find_all("strong")
    
    _p=converttolist(p)
    _em=converttolist(em)
    _bold=converttolist(bold)
    content = [ pr for pr in _p if pr not in _em and pr not in _bold]
    title=soup.head.title.string

    return title,content
    
    

In [106]:
# use conscine score to get the relevance/similarity between comments and article
import re, math
from collections import Counter

WORD = re.compile(r'\w+')
def get_cosine(vec1, vec2):
     intersection = set(vec1.keys()) & set(vec2.keys())
     numerator = sum([vec1[x] * vec2[x] for x in intersection])

     sum1 = sum([vec1[x]**2 for x in vec1.keys()])
     sum2 = sum([vec2[x]**2 for x in vec2.keys()])
     denominator = math.sqrt(sum1) * math.sqrt(sum2)

     if not denominator:
        return 0.0
     else:
        return float(numerator) / denominator

def text_to_vector(text):
     words = WORD.findall(text)
     return Counter(words)

# relevance score of each comment to whole aritle, and use the porpotion of each label to allocate
def article_avg_relevance_score(postid_file,file_name):
    urls=input_preparetion(postid_file)
    comments=comment_sentiment_score(file_name)
    final=pd.DataFrame()
    for i in range(urls.shape[0]):
        if pd.isnull(urls['Final Link'][i]):
            pass
        else :
            title,content=content_from_url(urls['Final Link'][i])
            mean_scores=pd.DataFrame()
            for j in range(comments.shape[0]):  
                if comments['Postid'][j]==urls['Postid'][i]:
                    total_score=[]
                    for w in range(len(content)):
                        text1 = comments['message'][j].lower()
                        text2 = content[w].lower()

                        vector1 = text_to_vector(text1)
                        vector2 = text_to_vector(text2)

                        cosine = get_cosine(vector1, vector2)
                        if cosine==0:
                            pass
                        else:
                            total_score.append(cosine)
                    if len(total_score)==0:
                        avg=0
                    else:
                        avg=mean(total_score)
                    comment_avg={'Index':j,'Avg':avg}
                    d=pd.DataFrame.from_dict(data=comment_avg,orient='index')
                    mean_scores=pd.concat([mean_scores,d],axis=1)
                else:
                    pass

            final=pd.concat([mean_scores.T,final])
    final1=final.set_index(final.Index.astype(int))['Avg']
    final2=comments.merge(final1, left_index=True, right_index=True,how='left', suffixes=('', '_y'))
    
    final_index=final2.groupby('Postid',as_index=False).mean()
    final_index['Positive_Comments_Relevent_Index']=final_index['Postive_Comments']*final_index['Avg']*100
    final_index['Neutral_Comments_Relevent_Index']=final_index['Neutral_Comments']*final_index['Avg']*100
    final_index['Negative_Comments_Relevent_Index']=final_index['Negative_Comments']*final_index['Avg']*100
    final_index=final_index[['Postid','Positive_Comments_Relevent_Index','Neutral_Comments_Relevent_Index','Negative_Comments_Relevent_Index']]
    final_index.sort_values(by=['Postid']).reset_index(drop=True)  
    
    x=sentiment_score(file_name).sort_values(by=['Postid']).reset_index(drop=True)  
    y=input_preparetion(postid_file).sort_values(by=['Postid']).reset_index(drop=True)
    
    z = y.merge(x,left_index=True, right_index=True,how='left', suffixes=('', '_y'))
    z=z.drop(['Postid_y'],axis=1)

    output = z.merge(final_index,left_index=True, right_index=True,how='left', suffixes=('', '_y'))
    output=output.drop(['Postid_y'],axis=1)
    return output


In [30]:
final=article_avg_relevance_score('postid_subset','comment_sample')

C:\Users\mezhang\AppData\Local\Anaconda\lib\site-packages\defusedxml\ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__init__(self, html, target, encoding)
C:\Users\mezhang\AppData\Local\Anaconda\lib\site-packages\defusedxml\ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__init__(self, html, target, encoding)
C:\Users\mezhang\AppData\Local\Anaconda\lib\site-packages\defusedxml\ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__init__(self, html, target, encoding)
C:\Users\mezhang\AppData\Local\Anaconda\lib\site-packages\defusedxml\ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__init__(self, html, target, encoding)
C:\Users\mezhang\AppData\Local\Anaconda\lib\site-packages\defusedxml\ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__in

In [102]:
# text preparation : remove stopwords, lemmatize word, remove unwanted words such as The, an, and
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True)) 


stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer() 
# stop_words.extend(["a", "an", "am", "is", "are"])
removelist=['CC','DT','FW','IN','LS','MD','PDT','POS','PRP','PRP$','RP','SYM','TO','WDT','WP','WP$','WRB']
# nltk.help.upenn_tagset()
def data_lemmatized(content):
    new_content=[]
    new_sentence=[]
    for sentence in content:
        word_type=nltk.pos_tag(sentence)
        keep_word=[word for word in word_type if word[1] not in removelist and word[1] not in stop_words]
        
        for i in range(len(keep_word)):
            new_sentence.append(lemmatizer.lemmatize(keep_word[i][0]))
    new_content.append(new_sentence)   
    return new_content

#level 2/3 top words
def big_tri_grams(content):
    sentence_stream = [doc.split(" ") for doc in content]
    bigram = Phrases(sentence_stream, min_count=1, delimiter=b' ')
    trigram = Phrases(bigram[sentence_stream], min_count=1, delimiter=b' ')
    bigrams_=[]
    trigrams_=[]
    for sent in sentence_stream:
        bigrams_.append([b for b in bigram[sent] if b.count(' ') == 1])
        trigrams_.append([t for t in trigram[bigram[sent]] if t.count(' ') == 2])
        
    b_flat = [ val  for sublist in bigrams_ for val in sublist]
    t_flat = [ val  for sublist in trigrams_ for val in sublist]
    
    b_flat1=[u for u in b_flat if b_flat.count(u) >1]
    t_flat1=[u for u in t_flat if t_flat.count(u) >1]
    
    b_final=[]
    t_final=[]
    
    for word in b_flat1:
        if word not in b_final:
            b_final.append(word)
            
            
    for word in t_flat1:
        if word not in t_final:
            t_final.append(word)
     
    b_=[item.title() for item in b_final if (nltk.pos_tag(item.split())[0][1] in ['NN','NNPS','NNP','NNS']) and 
                            (nltk.pos_tag(item.split())[1][1]  in ['NN','NNPS','NNP','NNS']) ]
    
    t_=[item.title() for item in t_final if (nltk.pos_tag(item.split())[0][1] in ['NN','NNPS','NNP','NNS']) and 
                             (nltk.pos_tag(item.split())[1][1]  in ['NN','NNPS','NNP','NNS']) and 
                                (nltk.pos_tag(item.split())[2][1]  in ['NN','NNPS','NNP','NNS'])]

    topwords_level2=",".join(b_)
    topwords_level3=",".join(t_)
    return topwords_level2,topwords_level3

#use LDA to get level 1 top words
def lda_model(data_words):
    xx=[]
    def get_list(nest_list):
        for item in nest_list:
            for i in item[1]:
                xx.append(i)
        return xx 

    texts = data_lemmatized(data_words)
    id2word = corpora.Dictionary(data_words)
    corpus = [id2word.doc2bow(text) for text in texts]
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
    doc_lda = lda_model[corpus]
    ldatopics = lda_model.show_topics(formatted=False)
    
    aa=pd.DataFrame.from_dict(get_list(ldatopics))
    aa.columns = ['Word', 'Prob']
    bb=pd.DataFrame.from_dict(nltk.pos_tag(aa['Word']))
    bb.columns = ['Word', 'Tag']
    cc=[aa.iloc[i,:] for i in range(bb.shape[0]) if bb['Tag'][i] in ['NN','NNPS','NNP','NNS']]
    dd=pd.DataFrame.from_dict(cc)
    ss=dd.groupby('Word',as_index=False).mean().sort_values(by='Prob',ascending=False)[:10]['Word']
    
    topwords_level1=",".join(ss.str.title())
    return topwords_level1
    
    

def lda_top_words(prepared_table):
    final=pd.DataFrame()
    for i in range (prepared_table.shape[0]):
        try:
            title,content=content_from_url(prepared_table['Final Link'][i])
            b_final,t_final=big_tri_grams(content)

            topwords_level1=lda_model(list(sent_to_words(content)))
            topwords_level2,topwords_level3=big_tri_grams(content)

            topwords=[topwords_level1,topwords_level2,topwords_level3]

            Dict = {'Postid':prepared_table['Postid'][i],'Topwords_level1': topwords_level1, 
                    'Topwords_level2': topwords_level2,'Topwords_level3': topwords_level3} 
            d=pd.DataFrame.from_dict(data=Dict,orient='index')
            final=pd.concat([final,d],axis=1)
        except:
            pass
    return final.T

In [64]:
zz=input_preparetion('filefromrati1')

C:\Users\mezhang\AppData\Local\Anaconda\lib\site-packages\defusedxml\ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__init__(self, html, target, encoding)
C:\Users\mezhang\AppData\Local\Anaconda\lib\site-packages\defusedxml\ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__init__(self, html, target, encoding)
C:\Users\mezhang\AppData\Local\Anaconda\lib\site-packages\defusedxml\ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__init__(self, html, target, encoding)
C:\Users\mezhang\AppData\Local\Anaconda\lib\site-packages\defusedxml\ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__init__(self, html, target, encoding)
C:\Users\mezhang\AppData\Local\Anaconda\lib\site-packages\defusedxml\ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__in

In [65]:
zz1=lda_top_words(zz)

In [67]:
final=article_avg_relevance_score('filefromrati1','filefromrati_comments')

C:\Users\mezhang\AppData\Local\Anaconda\lib\site-packages\defusedxml\ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__init__(self, html, target, encoding)
C:\Users\mezhang\AppData\Local\Anaconda\lib\site-packages\defusedxml\ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__init__(self, html, target, encoding)
C:\Users\mezhang\AppData\Local\Anaconda\lib\site-packages\defusedxml\ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__init__(self, html, target, encoding)
C:\Users\mezhang\AppData\Local\Anaconda\lib\site-packages\defusedxml\ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__init__(self, html, target, encoding)
C:\Users\mezhang\AppData\Local\Anaconda\lib\site-packages\defusedxml\ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__in

In [68]:

zz2=pd.concat([final.set_index('Postid'),zz1.set_index('Postid')], axis=1, join='outer').reset_index()

C:\Users\mezhang\AppData\Local\Anaconda\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [69]:
zz2

,index,Page Id,Fullid,Post_Headline,Final Link,Section,Subsection,Total_Comments,Postive_Comments,Neutral_Comments,Negative_Comments,NA_Comments,Positive_Comments_Relevent_Index,Neutral_Comments_Relevent_Index,Negative_Comments_Relevent_Index,Topwords_level1,Topwords_level2,Topwords_level3
0,10156952939723256,5953023255,5953023255_10156952939723256,Louis C.K. acknowledged in his comedy set that...,https://www.chicagotribune.com/entertainment/c...,entertainment,ct-ent-louis-ck-comedy-set-parkland-survivors-...,70.0,23.0,3.0,32.0,12.0,4.265397,0.556356,5.934465,"Club,Gender,Identity,People,Comedy,November,Yo...",Gender Identity,
1,10156953285703256,5953023255,5953023255_10156953285703256,“Why don’t we say those things to people while...,https://www.chicagotribune.com/lifestyles/stev...,lifestyles,stevens,13.0,8.0,0.0,1.0,4.0,7.640642,0.000000,0.955080,"Love,Day,People,Way,Kramer,Person,Life,Kind,Post","“I Didn’T,Kramer Wrote.",Kramer Wrote. “I
2,10156953862688256,5953023255,5953023255_10156953862688256,"Welcome to the world, Yarehlie Nava: Cook Coun...",https://www.chicagotribune.com/news/local/brea...,news,local,8.0,6.0,0.0,0.0,2.0,5.506126,0.000000,0.000000,"Ramirez,Baby,Yarehlie,Hour,Girl,County,Wasn","Baby Born,Medical Center",
3,10156953999383256,5953023255,5953023255_10156953999383256,The estate has a Cape Cod-style main house and...,https://www.chicagotribune.com/classified/real...,classified,realestate,10.0,3.0,0.0,1.0,6.0,3.092619,0.000000,1.030873,"Ali,Estate,House,Bedroom,Building,Property,Buy...","Gym Building,Lonnie Ali,",
4,10156954215838256,5953023255,5953023255_10156954215838256,There's still time for the baby boomers to red...,https://www.chicagotribune.com/news/opinion/ed...,news,opinion,40.0,15.0,0.0,16.0,9.0,5.884323,0.000000,6.276611,"Boomer,Time,World,Generation,Baby,Sorry,Someone",,
5,10156954402858256,5953023255,5953023255_10156954402858256,Nick Foles is replacing Carson Wentz at quarte...,https://www.chicagotribune.com/sports/football...,sports,football,11.0,4.0,0.0,3.0,4.0,3.870788,0.000000,2.903091,"Bowl,Year,Game,Season,Playoff,Quarterback,Supe...","Super Bowl,Nfl Record,Quarterback Hits,",
6,10156954618048256,5953023255,5953023255_10156954618048256,The brother/sister duo of Francis Scott Key Wh...,https://www.chicagotribune.com/entertainment/m...,entertainment,music,4.0,1.0,0.0,0.0,3.0,0.868508,0.000000,0.000000,"Francis,Chicago,Song,Band,Year,Alex,Bear,Red,M...","Miss Alex,Chicago Bears",Miss Alex White
7,10156954909978256,5953023255,5953023255_10156954909978256,"A sellout crowd of 76,126 made this year's Win...",https://www.chicagotribune.com/sports/hockey/b...,sports,hockey,7.0,2.0,0.0,1.0,4.0,4.502885,0.000000,2.251442,"Hawk,Toews,Dame,Game,Center,Year,Notre,Second,...","Dame Stadium,“I Think,Winter Classic.,Notre Da...",
8,10156954957663256,5953023255,5953023255_10156954957663256,The new year has already brought a new milesto...,https://www.chicagotribune.com/news/nationworl...,news,nationworld,3.0,1.0,0.0,1.0,1.0,3.745589,0.000000,3.745589,"Apl,Scientist,Mission,Thule,Ultima,Object,Spac...","Nasa Shows,New Horizons,Kuiper Belt,,Mission O...",New Horizons Spacecraft.
9,10156955515998256,5953023255,5953023255_10156955515998256,Jerry Falwell Jr. was asked if there is anythi...,https://www.chicagotribune.com/news/nationworl...,news,nationworld,125.0,39.0,2.0,46.0,38.0,5.053698,0.259164,5.960772,"President,Kingdom,Country,Falwell,Think,Jesus,...","Liberty University,President Jerry,Falwell Jr....",


In [5]:
result=input_preparetion('filefromrati1')
result['Fullid'].to_csv('post')
summary=sentiment_score('post1')

C:\Users\mezhang\AppData\Local\Anaconda\lib\site-packages\defusedxml\ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__init__(self, html, target, encoding)
C:\Users\mezhang\AppData\Local\Anaconda\lib\site-packages\defusedxml\ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__init__(self, html, target, encoding)
C:\Users\mezhang\AppData\Local\Anaconda\lib\site-packages\defusedxml\ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__init__(self, html, target, encoding)
C:\Users\mezhang\AppData\Local\Anaconda\lib\site-packages\defusedxml\ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__init__(self, html, target, encoding)
C:\Users\mezhang\AppData\Local\Anaconda\lib\site-packages\defusedxml\ElementTree.py:68: DeprecationWarning: The html argument of XMLParser() is deprecated
  _XMLParser.__in

In [70]:
zz2.to_excel("fullpost_index.xlsx",sheet_name='Sheet1')

C:\Users\mezhang\AppData\Local\Anaconda\lib\site-packages\xlsxwriter\worksheet.py:912: UserWarning: Ignoring URL 'https://www.chicagotribune.com/lifestyles/ct-life-soc-polar-vortex-dogs-20190131-story.html?fbclid=IwAR0_LmDoMVQaY2W3RsHzWmJ_ZxJfu-AsbRJ-piGYIR4-V4tvhvhy-oJ886s&fbclid=IwAR0r_SE29QBnBzvqDi2OWOqlon5hQ5q-x-Rc__pW04RFvALaRDIglNoXncI&fbclid=IwAR1nNelErDxX5SoC1DtXVNjLtLsCXRdielA4ATf8MMx2aX4i7UVC5eaWOVk' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))


In [60]:
subset=result.iloc[:20,:]


subset

,Page Id,Postid,Fullid,Post_Headline,Final Link,Section,Subsection
0,5953023255,10157024122548256,5953023255_10157024122548256,Someone has paid for hotel rooms on the South ...,https://www.chicagotribune.com/news/weather/ct...,news,weather
1,5953023255,10157030126903256,5953023255_10157030126903256,We earned it.,https://www.chicagotribune.com/news/weather/ct...,news,weather
2,5953023255,10157022568463256,5953023255_10157022568463256,Some Chicagoans don't have the option of worki...,https://www.chicagotribune.com/news/opinion/hu...,news,opinion
3,5953023255,10156987725358256,5953023255_10156987725358256,"“I could care less about college football,” Al...",https://www.chicagotribune.com/dining/ct-food-...,dining,ct-food-alinea-clemson-tigers-trump-20190116-s...
4,5953023255,10157087764758256,5953023255_10157087764758256,Pet them. Feed them. Photo ops galore. Here's ...,https://www.chicagotribune.com/entertainment/m...,entertainment,museums
5,5953023255,10157026581353256,5953023255_10157026581353256,A group of about eight friends — business owne...,https://www.chicagotribune.com/news/weather/ct...,news,weather
6,5953023255,10157057616613256,5953023255_10157057616613256,The caller told the 90-year-old Washington man...,https://www.chicagotribune.com/business/ct-biz...,business,ct-biz-jamaican-phone-scam-william-webster-201...
7,5953023255,10156991793308256,5953023255_10156991793308256,After helping the Bears make a worst-to-first ...,https://www.chicagotribune.com/sports/football...,sports,football
8,5953023255,10157020475343256,5953023255_10157020475343256,"The CTA tracks may look like a scene out of ""G...",https://www.chicagotribune.com/news/local/brea...,news,local
9,5953023255,10157045107508256,5953023255_10157045107508256,A proposal to legalize recreational marijuana ...,https://www.chicagotribune.com/news/ct-met-ill...,news,ct-met-illinois-legalize-recreational-marijuan...


In [65]:
y.to_excel("topwords.xlsx",sheet_name='Sheet1')

In [336]:
b_final,t_final=big_tri_grams(x1)
b_final

'Indiana Dunes,National Park,South Shore'

In [314]:
sentence_stream = [doc.split(" ") for doc in subset]
bigram = Phrases(sentence_stream, min_count=1, delimiter=b' ')
trigram = Phrases(bigram[sentence_stream], min_count=1, delimiter=b' ')
bigrams_=[]
trigrams_=[]

for sent in sentence_stream:
    bigrams_.append([b for b in bigram[sent] if b.count(' ') == 1])
    trigrams_.append([t for t in trigram[bigram[sent]] if t.count(' ') == 2])

b_flat = [ val  for sublist in bigrams_ for val in sublist]
t_flat = [ val  for sublist in trigrams_ for val in sublist]

b_flat1=[u for u in b_flat if b_flat.count(u) >1]
t_flat1=[u for u in t_flat if t_flat.count(u) >1]

b_final=[]
t_final=[]

In [312]:
[b for b in bigram[sent] if b.count(' ') == 1]

[['Page', 'Id'],
 ['Postid'],
 ['Fullid'],
 ['Post_Headline'],
 ['Final', 'Link'],
 ['Section'],
 ['Subsection']]

In [288]:
subset1=pd.concat([subset,final],axis=1)
subset1

,Page Id,Postid,Fullid,Post_Headline,Final Link,Section,Subsection,Topwords_level1,Topwords_level2,Topwords_level3
0,5953023255,10157024122548256,5953023255_10157024122548256,Someone has paid for hotel rooms on the South ...,https://www.chicagotribune.com/news/weather/ct...,news,weather,"Propane,Rachev,Hotel,Dozen,Center,People,Camp,...",Propane Tanks,
1,5953023255,10157030126903256,5953023255_10157030126903256,We earned it.,https://www.chicagotribune.com/news/weather/ct...,news,weather,"Groundhog,Handler,Rd,Punxsutawney,Gobbler,Day,...","Groundhog Club,Co-Handler John,Griffiths, Left...",
2,5953023255,10157022568463256,5953023255_10157022568463256,Some Chicagoans don't have the option of worki...,https://www.chicagotribune.com/news/opinion/hu...,news,opinion,"Making,Day,Home,Cold,Face,Don,World,Grateful,Wind",,
3,5953023255,10156987725358256,5953023255_10156987725358256,"“I could care less about college football,” Al...",https://www.chicagotribune.com/dining/ct-food-...,dining,ct-food-alinea-clemson-tigers-trump-20190116-s...,"Clemson,Kokonas,Alinea,Meal,Football,Burger,Re,Ll","Clemson Tigers,White House,Burger King",
4,5953023255,10157087764758256,5953023255_10157087764758256,Pet them. Feed them. Photo ops galore. Here's ...,https://www.chicagotribune.com/entertainment/m...,entertainment,museums,"Aquarium,Sea,Shedd,Experience,Otter,Luna,Encou...",Shedd Aquarium,
5,5953023255,10157026581353256,5953023255_10157026581353256,A group of about eight friends — business owne...,https://www.chicagotribune.com/news/weather/ct...,news,weather,"Hotel,Payne,Room,Night,People,Tent,Homeless,Ca...","Thursday Afternoon,,Homeless Camp,Dan Ryan,Pro...",
6,5953023255,10157057616613256,5953023255_10157057616613256,The caller told the 90-year-old Washington man...,https://www.chicagotribune.com/business/ct-biz...,business,ct-biz-jamaican-phone-scam-william-webster-201...,"Told,Thomas,Webster,Call,Man,Year,Victim,Lynda...","Washington Man,William Webster,Lynda Webster,""...",
7,5953023255,10156991793308256,5953023255_10156991793308256,After helping the Bears make a worst-to-first ...,https://www.chicagotribune.com/sports/football...,sports,football,"Coach,Year,Pfwa,Smith,Team,Season,Nagy,Playoff...",Nfl Coach,


In [287]:
final=lda_top_words(subset)

In [268]:
 nltk.tag.pos_tag(['were'])

[('were', 'VBD')]

In [143]:
title,content=content_from_url("https://www.chicagotribune.com/suburbs/post-tribune/news/ct-ptb-indiana-dunes-national-park-st-0219-story.html?fbclid=IwAR10I-PV_OsnmyuEEoYbSNIl_kscTM4Cj3NxkOyesho5sWQ4nx6fnc029L8")

In [265]:

def lda_model(data_words):
    xx=[]
    def get_list(nest_list):
        for item in nest_list:
            for i in item[1]:
                xx.append(i)
        return xx 

    texts = data_lemmatized(data_words)
    id2word = corpora.Dictionary(data_words)
    corpus = [id2word.doc2bow(text) for text in texts]
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
    doc_lda = lda_model[corpus]
    ldatopics = lda_model.show_topics(formatted=False)
    
    aa=pd.DataFrame.from_dict(get_list(ldatopics))
    aa.columns = ['Word', 'Prob']
    bb=pd.DataFrame.from_dict(nltk.pos_tag(aa['Word']))
    bb.columns = ['Word', 'Tag']
    cc=[aa.iloc[i,:] for i in range(bb.shape[0]) if bb['Tag'][i] in ['NN','NNPS','NNP','NNS']]
    dd=pd.DataFrame.from_dict(cc)
    ss=dd.groupby('Word',as_index=False).mean().sort_values(by='Prob',ascending=False)[:10]['Word']
    
    topwords_level1=",".join(ss.str.title())
    return topwords_level1
    
    

In [232]:
title,content=content_from_url(subset['Final Link'][0])


In [233]:
 
x=lda_model(list(sent_to_words(content)) )
x

'propane/rachev/hotel/dozen/center/people/camp/city/women'

In [165]:
data_lemmatized(data_words)

id2word = corpora.Dictionary(data_lemmatized(data_words))

# Create Corpus
texts = data_lemmatized(data_words)

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

doc_lda = lda_model[corpus]
ldatopics = lda_model.show_topics(formatted=False)
ldatopics


aa=pd.DataFrame.from_dict(get_list(ldatopics))
aa.columns = ['Word', 'Prob']
bb=pd.DataFrame.from_dict(nltk.pos_tag(aa['Word']))
bb.columns = ['Word', 'Tag']
cc=[aa.iloc[i,:] for i in range(bb.shape[0]) if bb['Tag'][i] in ['NN','NNPS','NNP','NNS']]
dd=pd.DataFrame.from_dict(cc)
ss=dd.groupby('Word',as_index=False).mean().sort_values(by='Prob',ascending=False)[:10]['Word']

topwords_level1="/".join(ss)

topwords_level1

'park/change/indiana/dune/state/visitor/people/year'

In [16]:

        
        
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer() 
# stop_words.extend(["a", "an", "am", "is", "are"])
removelist=['CC','DT','FW','IN','LS','MD','PDT','POS','PRP','PRP$','RP','SYM','TO','WDT','WP','WP$','WRB']
# nltk.help.upenn_tagset()
def data_lemmatized(content):
    new_content=[]
    new_sentence=[]
    for sentence in content:
        word_type=nltk.pos_tag(sentence)
        keep_word=[word for word in word_type if word[1] not in removelist and word[1] not in stop_words]
        
        for i in range(len(keep_word)):
            new_sentence.append(lemmatizer.lemmatize(keep_word[i][0]))
    new_content.append(new_sentence)   
    return new_content



In [115]:
def big_tri_grams(content):
    sentence_stream = [doc.split(" ") for doc in content]
    bigram = Phrases(sentence_stream, min_count=1, delimiter=b' ')
    trigram = Phrases(bigram[sentence_stream], min_count=1, delimiter=b' ')
    bigrams_=[]
    trigrams_=[]
    for sent in sentence_stream:
        bigrams_.append([b for b in bigram[sent] if b.count(' ') == 1])
        trigrams_.append([t for t in trigram[bigram[sent]] if t.count(' ') == 2])
        
    b_flat = [ val.title()  for sublist in bigrams_ for val in sublist]
    t_flat = [ val.title()  for sublist in trigrams_ for val in sublist]
    
    b_flat1=[u for u in b_flat if b_flat.count(u) >1]
    t_flat1=[u for u in t_flat if t_flat.count(u) >1]
    
    b_final=[]
    t_final=[]
    
    for word in b_flat1:
        if word not in b_final:
            b_final.append(word)
            
            
    for word in t_flat1:
        if word not in t_final:
            t_final.append(word)
     
    b_=[item for item in b_final if (nltk.pos_tag(item.split())[0][1] in ['NN','NNPS','NNP','NNS']) and 
                            (nltk.pos_tag(item.split())[1][1]  in ['NN','NNPS','NNP','NNS']) ]
    
    t_=[item for item in t_final if (nltk.pos_tag(item.split())[0][1] in ['NN','NNPS','NNP','NNS']) and 
                            (nltk.pos_tag(item.split())[1][1]  in ['NN','NNPS','NNP','NNS']) ]


    return b_,t_

In [117]:
b_final,t_final=big_tri_grams(content)
t_final

[]

In [134]:
dd="/".join(b_final)
dd

'Indiana Dunes/National Park/National Parks./South Shore'

In [120]:
def content_normalization(url):
    title,content=content_from_url(url)
    splited_content=[]
    for line in content:
        pr=re.sub(r"[,.;@#?!&$]+", ' ', line) 
        _pr = re.sub(r" ", ' ', pr) 
        split_text=re.split(r' ' , _pr)
        final_text=list(filter(None,split_text))
        stop_words = set(stopwords.words('english'))
        final=[ w.lower().replace('“', "")  for w in final_text if not w.lower().replace('“', "") in stop_words]
        splited_content=splited_content+final
    return splited_content
                                

In [489]:
result=input_preparetion('filefromrati1')
result

,Page Id,Postid,Fullid,Post_Headline,Final Link,Section,Subsection
0,5953023255,10157024122548256,5953023255_10157024122548256,Someone has paid for hotel rooms on the South ...,https://www.chicagotribune.com/news/weather/ct...,news,weather
1,5953023255,10157030126903256,5953023255_10157030126903256,We earned it.,https://www.chicagotribune.com/news/weather/ct...,news,weather
2,5953023255,10157022568463256,5953023255_10157022568463256,Some Chicagoans don't have the option of worki...,https://www.chicagotribune.com/news/opinion/hu...,news,opinion
3,5953023255,10156987725358256,5953023255_10156987725358256,"“I could care less about college football,” Al...",https://www.chicagotribune.com/dining/ct-food-...,dining,ct-food-alinea-clemson-tigers-trump-20190116-s...
4,5953023255,10157087764758256,5953023255_10157087764758256,Pet them. Feed them. Photo ops galore. Here's ...,https://www.chicagotribune.com/entertainment/m...,entertainment,museums
5,5953023255,10157026581353256,5953023255_10157026581353256,A group of about eight friends — business owne...,https://www.chicagotribune.com/news/weather/ct...,news,weather
6,5953023255,10157057616613256,5953023255_10157057616613256,The caller told the 90-year-old Washington man...,https://www.chicagotribune.com/business/ct-biz...,business,ct-biz-jamaican-phone-scam-william-webster-201...
7,5953023255,10156991793308256,5953023255_10156991793308256,After helping the Bears make a worst-to-first ...,https://www.chicagotribune.com/sports/football...,sports,football
8,5953023255,10157020475343256,5953023255_10157020475343256,"The CTA tracks may look like a scene out of ""G...",https://www.chicagotribune.com/news/local/brea...,news,local
9,5953023255,10157045107508256,5953023255_10157045107508256,A proposal to legalize recreational marijuana ...,https://www.chicagotribune.com/news/ct-met-ill...,news,ct-met-illinois-legalize-recreational-marijuan...


In [327]:
x=content_normalization(result['Final Link'][1])
x

['groundhog',
 'club',
 'co-handler',
 'john',
 'griffiths',
 'left',
 'holds',
 'punxsutawney',
 'phil',
 'weather',
 'prognosticating',
 'groundhog',
 '133rd',
 'celebration',
 'groundhog',
 'day',
 "gobbler's",
 'knob',
 'punxsutawney',
 'pa',
 'feb',
 '2',
 '2019',
 'groundhog',
 'club',
 'co-handler',
 'john',
 'griffiths',
 'left',
 'holds',
 'punxsutawney',
 'phil',
 'weather',
 'prognosticating',
 'groundhog',
 '133rd',
 'celebration',
 'groundhog',
 'day',
 "gobbler's",
 'knob',
 'punxsutawney',
 'pa',
 'feb',
 '2',
 '2019',
 'may',
 'hard',
 'believe',
 'large',
 'swath',
 'u',
 'thaws',
 'bitter',
 'polar',
 'vortex',
 'spring',
 'coming',
 'early',
 'according',
 'handlers',
 "country's",
 'famous',
 'prognosticating',
 'groundhogs',
 '7:30',
 'saturday',
 'punxsutawney',
 "(pronounced\xa0puhnk-suh-taw'-nee)",
 'phil',
 'emerged',
 'burrow',
 'pennsylvania',
 'sunrise',
 'see',
 'shadow',
 'nearly',
 'series',
 'events',
 'unfolded',
 '300',
 'miles',
 'east',
 'staten',
 '

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True)) 
 
data_words = list(sent_to_words(content))  
data_words

In [320]:
def top_nlevel_words(normalized_content, level, topn):
    vec1 = CountVectorizer(ngram_range=(level,level),  
            max_features=2000).fit(normalized_content)
    bag_of_words = vec1.transform(normalized_content)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in     
                  vec1.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], 
                reverse=True)
    top_df = pd.DataFrame(words_freq[:topn])
    top_df.columns=["Word", "Freq"]
    keywords=[]
    for j in range(topn):
            keywords.append(top_df["Word"][j])
    return top_df,keywords
    

In [460]:
def topwords_forpostid(comment_file):
    link=comment_file['Final Link'].replace(np.nan, '', regex=True)
    blank={}
    for i in range(len(link)):
        if link[i]=='':
            top={i:[]}
        else:
            x=content_normalization(link[i])
            top_df,keywords=top_nlevel_words(x, 1, 10)
            top={i:keywords}
        blank.update(top) 
    final=pd.DataFrame.from_dict(blank,orient='index') 
    combine=pd.merge(comment_file, final, left_index=True,right_index=True,how='left')
    return combine

In [490]:
x=topwords_forpostid(result)

,Page Id,Postid,Fullid,Post_Headline,Final Link,Section,Subsection,0,1,2,3,4,5,6,7,8,9
0,5953023255,10157024122548256,5953023255_10157024122548256,Someone has paid for hotel rooms on the South ...,https://www.chicagotribune.com/news/weather/ct...,news,weather,hotel,propane,tanks,said,people,good,samaritan,confiscated,camp,rachev
1,5953023255,10157030126903256,5953023255_10157030126903256,We earned it.,https://www.chicagotribune.com/news/weather/ct...,news,weather,groundhog,punxsutawney,phil,day,gobbler,knob,feb,club,co,handler
2,5953023255,10157022568463256,5953023255_10157022568463256,Some Chicagoans don't have the option of worki...,https://www.chicagotribune.com/news/opinion/hu...,news,opinion,cold,it,day,grateful,world,around,essential,workers,vortex,home
3,5953023255,10156987725358256,5953023255_10156987725358256,"“I could care less about college football,” Al...",https://www.chicagotribune.com/dining/ct-food-...,dining,ct-food-alinea-clemson-tigers-trump-20190116-s...,kokonas,alinea,clemson,food,like,would,football,restaurant,dinner,fast
4,5953023255,10157087764758256,5953023255_10157087764758256,Pet them. Feed them. Photo ops galore. Here's ...,https://www.chicagotribune.com/entertainment/m...,entertainment,museums,shedd,otter,sea,like,animals,otters,luna,aquarium,people,sterling
5,5953023255,10157026581353256,5953023255_10157026581353256,A group of about eight friends — business owne...,https://www.chicagotribune.com/news/weather/ct...,news,weather,payne,said,homeless,friends,people,camp,temperatures,room,inn,get
6,5953023255,10157057616613256,5953023255_10157057616613256,The caller told the 90-year-old Washington man...,https://www.chicagotribune.com/business/ct-biz...,business,ct-biz-jamaican-phone-scam-william-webster-201...,thomas,webster,man,000,one,told,said,jamaican,money,fbi
7,5953023255,10156991793308256,5953023255_10156991793308256,After helping the Bears make a worst-to-first ...,https://www.chicagotribune.com/sports/football...,sports,football,coach,bears,first,named,nfl,year,pfwa,nagy,team,award
8,5953023255,10157020475343256,5953023255_10157020475343256,"The CTA tracks may look like a scene out of ""G...",https://www.chicagotribune.com/news/local/brea...,news,local,rail,metra,switches,fire,system,along,winter,cold,weather,breaks
9,5953023255,10157045107508256,5953023255_10157045107508256,A proposal to legalize recreational marijuana ...,https://www.chicagotribune.com/news/ct-met-ill...,news,ct-met-illinois-legalize-recreational-marijuan...,marijuana,bill,said,ammons,would,state,illinois,percent,cassidy,steans


In [491]:
x.head()

AttributeError: 'list' object has no attribute 'head'

In [466]:

for i in range(len(link)):
    if link[i]=='':
        top={i:[]}
    else:
        x=content_normalization(link[i])
        top_df,keywords=top_nlevel_words(x, 1, 10)
        top={i:keywords}
        print(top)


{0: ['hotel', 'propane', 'tanks', 'said', 'people', 'good', 'samaritan', 'confiscated', 'camp', 'rachev']}
{1: ['groundhog', 'punxsutawney', 'phil', 'day', 'gobbler', 'knob', 'feb', 'club', 'co', 'handler']}
{4: ['shedd', 'otter', 'sea', 'like', 'animals', 'otters', 'luna', 'aquarium', 'people', 'sterling']}
{5: ['payne', 'said', 'homeless', 'friends', 'people', 'camp', 'temperatures', 'room', 'inn', 'get']}
{8: ['rail', 'metra', 'switches', 'fire', 'system', 'along', 'winter', 'cold', 'weather', 'breaks']}
{11: ['obama', 'music', 'us', 'every', 'lady', 'appearance', 'audience', 'story', 'within', 'michelle']}
{13: ['park', 'said', 'national', 'indiana', 'dunes', 'parks', 'state', 'change', 'visitors', 'added']}


In [430]:
link=result['Final Link'].replace(np.nan, '', regex=True)
blank=pd.DataFrame([])
blank

""


In [444]:
comment_file=pd.concat([comment_file,top],ignore_index=False,axis=1) 

NameError: name 'comment_file' is not defined

In [467]:
topwords_forpostid(result).to_excel('topwords.xlsx',sheet_name='top')

In [186]:
from sklearn.feature_extraction.text import TfidfTransformer
 
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)
tf_idf_vector=tfidf_transformer.transform(cv.transform(x))

def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    #get the feature names and tf-idf score of top n items
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]
 
    score_vals = []
    feature_vals = []
    
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])
 
    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results


sorted_items=sort_coo(tf_idf_vector.tocoo())

#extract only the top n; n here is 10
keywords=extract_topn_from_vector(feature_names,sorted_items,5)

keywords

{'zero': 1.0, 'wonderful': 1.0, 'women': 1.0, 'without': 1.0}

In [185]:
from sklearn.feature_extraction.text import TfidfTransformer
 
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)
feature_names=cv.get_feature_names()
tf_idf_vector=tfidf_transformer.transform(cv.transform(x))
tf_idf_vector

<242x172 sparse matrix of type '<class 'numpy.float64'>'
	with 237 stored elements in Compressed Sparse Row format>

In [176]:
def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]
 
    score_vals = []
    feature_vals = []
    
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        
        feature_vals.append(feature_names[idx])
 
    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results
 

In [177]:
feature_names=cv.get_feature_names()
Keywords=extract_topn_from_vector(feature_names,sorted_items,10)
print("\n===Keywords===")
for k in Keywords:
    print(k,Keywords[k])
 


===Keywords===
zero 1.0
wonderful 1.0
women 1.0
without 1.0
willis 1.0
week 1.0
wednesday 1.0
